* **KNN funciona bien en dataset con un pequeño número de características/columnas.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor

# Ejercicio 1

Realizar un clasificador de películas de cine, tomando como referencia un dataset con una descripción de la película.


In [ ]:
filename = "/content/reviews_sentiment2.xlsx"
df = pd.read_excel(filename)
df_copy= df
df.head(10)

,Review Title,Review Text,wordcount,titleSentiment,textSentiment,Star Rating,sentimentValue
0,Sin conexiÃ³n,Hola desde hace algo mÃ¡s de un mes me pone si...,23,negative,negative,1,-0.486389
1,faltan cosas,Han mejorado la apariencia pero no,20,negative,negative,1,-0.586187
2,Es muy buena lo recomiendo,Andres e puto amoooo,4,NaN,negative,1,-0.602240
3,Version antigua,Me gustana mas la version anterior esta es mas...,17,NaN,negative,1,-0.616271
4,Esta bien,Sin ser la biblia.... Esta bien,6,negative,negative,1,-0.651784
5,Buena,Nada del otro mundo pero han mejorado mucho,8,positive,negative,1,-0.720443
6,De gran ayuda,"Lo malo q necesita de â€¦,pero la app es muy b...",23,positive,negative,1,-0.726825
7,Muy buena,"Estaba mÃ¡s acostumbrado al otro diseÃ±o, pero...",16,positive,negative,1,-0.736769
8,Ta to guapa.,Va de escÃ¡ndalo,21,positive,negative,1,-0.765284
9,Se han corregido,Han corregido muchos fallos pero el diseÃ±o es...,13,negative,negative,1,-0.797961


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Review Title    257 non-null    object 
 1   Review Text     257 non-null    object 
 2   wordcount       257 non-null    int64  
 3   titleSentiment  231 non-null    object 
 4   textSentiment   257 non-null    object 
 5   Star Rating     257 non-null    int64  
 6   sentimentValue  257 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 14.2+ KB


In [ ]:
df.isna().sum()

Review Title       0
Review Text        0
wordcount          0
titleSentiment    26
textSentiment      0
Star Rating        0
sentimentValue     0
dtype: int64

El dataset posee valores nulos en la columna "titleSentiment"

In [ ]:
df[df["titleSentiment"]=="negative"]["textSentiment"].value_counts()

negative    21
positive    15
Name: textSentiment, dtype: int64

In [ ]:
df[df["titleSentiment"]=="positive"]["textSentiment"].value_counts()

positive    174
negative     21
Name: textSentiment, dtype: int64

In [ ]:
df[df["textSentiment"]=="negative"]["titleSentiment"].value_counts()

negative    21
positive    21
Name: titleSentiment, dtype: int64

In [ ]:
df[df["textSentiment"]=="positive"]["titleSentiment"].value_counts()

positive    174
negative     15
Name: titleSentiment, dtype: int64

No se ve relación entre ambas columnas.

In [ ]:
df[df["titleSentiment"].isnull()]

,Review Title,Review Text,wordcount,titleSentiment,textSentiment,Star Rating,sentimentValue
2,Es muy buena lo recomiendo,Andres e puto amoooo,4,NaN,negative,1,-6.022403e-01
3,Version antigua,Me gustana mas la version anterior esta es mas...,17,NaN,negative,1,-6.162712e-01
30,"Buena app, pero","Buena app, peroâ€¦",76,NaN,negative,2,-2.951937e-01
41,Util,"Funciona bien,â€¦",9,NaN,positive,3,-1.203939e-02
61,Me gusta,Es una buena herramienta paraâ€¦,10,NaN,positive,3,-1.109670e-01
67,ðŸ‘,ðŸ˜‚ðŸ‘,1,NaN,positive,3,-1.398608e-01
68,BuenÃ­sima,"A.,,",1,NaN,positive,3,-1.398608e-01
83,Una pena,La nueva versiÃ³n â€¦ de la vaca.,7,NaN,negative,1,-3.968495e-01
90,Muy buena,Buenas tardes necesito saber como hago para qu...,20,NaN,positive,3,1.545803e-03
102,Buena aplicaciÃ³n,Sitio donde,4,NaN,positive,3,1.251262e-01


In [ ]:
# for i in df.index:
#   if df["titleSentiment"][i].isna():
#     df["titleSentiment"][i]="indeterminado"

# df["titleSentiment"].value_counts()

AttributeError: ignored

In [ ]:
df["titleSentiment"].fillna("indeterminado", inplace = True) 
df["titleSentiment"].value_counts()

positive         195
negative          36
indeterminado     26
Name: titleSentiment, dtype: int64

Se rellenan los valores nulos de la columna "titleSentiment" con la característica indeterminado, para poder conservar los datos, ya que no se pudo inferir una relación con otras columnas.

In [ ]:
df.duplicated().sum()

1

Se encuentra un valor duplicado.

In [ ]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 256
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Review Title    256 non-null    object 
 1   Review Text     256 non-null    object 
 2   wordcount       256 non-null    int64  
 3   titleSentiment  256 non-null    object 
 4   textSentiment   256 non-null    object 
 5   Star Rating     256 non-null    int64  
 6   sentimentValue  256 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 16.0+ KB


Se elimina el valor duplicado.

In [ ]:
df["titleSentiment"].value_counts()

positive         194
negative          36
indeterminado     26
Name: titleSentiment, dtype: int64

In [ ]:
df["textSentiment"].value_counts()

positive    210
negative     46
Name: textSentiment, dtype: int64

In [ ]:
for i in df.index:
  if df["titleSentiment"][i]=="positive":
    df["titleSentiment"][i]=2
  elif df["titleSentiment"][i]=="negative":
    df["titleSentiment"][i]=1
  elif df["titleSentiment"][i]=="indeterminado":
    df["titleSentiment"][i]=0

df["titleSentiment"].value_counts()

In [ ]:
for i in df.index:
  if df["textSentiment"][i]=="positive":
    df["textSentiment"][i]=2
  elif df["textSentiment"][i]=="negative":
    df["textSentiment"][i]=1


df["textSentiment"].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2    210
1     46
Name: textSentiment, dtype: int64

Se transforman variables categóricas a númericas para poder operar con ellas.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 256
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Review Title    256 non-null    object 
 1   Review Text     256 non-null    object 
 2   wordcount       256 non-null    int64  
 3   titleSentiment  256 non-null    object 
 4   textSentiment   256 non-null    object 
 5   Star Rating     256 non-null    int64  
 6   sentimentValue  256 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 24.1+ KB


In [ ]:
df = df.drop(columns=["Review Title", "Review Text"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 256
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   wordcount       256 non-null    int64  
 1   titleSentiment  256 non-null    object 
 2   textSentiment   256 non-null    object 
 3   Star Rating     256 non-null    int64  
 4   sentimentValue  256 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 20.1+ KB


Se eliminan columnas tipo objeto, que no serán usadas para el ejercicio.

In [ ]:
df["textSentiment"] = df["textSentiment"].astype(int)
df["titleSentiment"] = df["titleSentiment"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 0 to 256
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   wordcount       256 non-null    int64  
 1   titleSentiment  256 non-null    int64  
 2   textSentiment   256 non-null    int64  
 3   Star Rating     256 non-null    int64  
 4   sentimentValue  256 non-null    float64
dtypes: float64(1), int64(4)
memory usage: 20.1 KB


Se transforman las variables tipo objeto a enteros.

In [ ]:
X = df.drop(columns=["Star Rating"])
y = df["Star Rating"]

X

,wordcount,titleSentiment,textSentiment,sentimentValue
0,23,1,1,-4.863893e-01
1,20,1,1,-5.861875e-01
2,4,0,1,-6.022403e-01
3,17,0,1,-6.162712e-01
4,6,1,1,-6.517836e-01
...,...,...,...,...
252,3,2,2,2.814818e+16
253,1,2,2,2.924393e+16
254,1,2,2,2.924393e+16
255,1,2,2,2.924393e+16


In [ ]:
df["Star Rating"].value_counts()

5    87
3    78
1    37
4    30
2    24
Name: Star Rating, dtype: int64

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,test_size=0.5)

In [ ]:
print("Muestras de entrenamiento", X_train.shape)
print("Muestras de validacion", X_val.shape)
print("Muestras de test", X_test.shape)

Muestras de entrenamiento (179, 4)
Muestras de validacion (38, 4)
Muestras de test (39, 4)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val =scaler.transform(X_val)

In [ ]:
acc=0
k=1
aux=0
aux1=0
while(acc<=0.7):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  predictions = knn.predict(X_test)
  acc=knn.score(X_test, y_test)
  if (acc==aux):
    aux1=aux1+1
  aux=acc
  if (aux1==5):
    k=k+1
    aux1=0
  if (k==len(X_train)):
    break
  print(acc)

0.717948717948718


In [ ]:
validacion=knn.predict(X_val)

In [ ]:
print("Valores predichos", validacion.tolist())
print("Valores reales", y_val.tolist())
print('Accuracy:',knn.score(X_val, y_val))

Valores predichos [2, 3, 5, 2, 5, 3, 3, 1, 5, 2, 2, 4, 5, 5, 5, 5, 3, 3, 3, 1, 3, 5, 2, 5, 1, 2, 3, 4, 4, 3, 2, 2, 5, 3, 3, 3, 2, 1]
Valores reales [2, 3, 5, 3, 5, 5, 3, 1, 4, 2, 2, 5, 5, 3, 4, 3, 3, 5, 3, 1, 3, 4, 4, 5, 1, 2, 1, 4, 4, 4, 2, 2, 3, 3, 5, 5, 2, 1]
Accuracy: 0.6052631578947368


In [ ]:
print(classification_report(y_val, validacion, target_names=["1", "2", "3", "4", "5"]))

              precision    recall  f1-score   support

           1       1.00      0.80      0.89         5
           2       0.78      1.00      0.88         7
           3       0.50      0.60      0.55        10
           4       0.67      0.29      0.40         7
           5       0.40      0.44      0.42         9

    accuracy                           0.61        38
   macro avg       0.67      0.63      0.63        38
weighted avg       0.62      0.61      0.60        38



Para clases desbalanceadas, en vez de optimizar el r2, se debe optimizar el recall.

# Ejercicio 2

Generar modelo para predecir "Type 1" en base a las características de pelea de los pokemon.

In [ ]:
filename = "/content/pokemon.csv"
df = pd.read_csv(filename)
df_copy= df
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [ ]:
df.isna().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [ ]:
df = df.drop(columns=["Type 2"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Total       800 non-null    int64 
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(2)
memory usage: 69.7+ KB


Se elimina la columna "Type 2" ya que poseía muchos valores nulos.

In [ ]:
df = df.drop(columns=["Name", "Legendary", "#"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Type 1      800 non-null    object
 1   Total       800 non-null    int64 
 2   HP          800 non-null    int64 
 3   Attack      800 non-null    int64 
 4   Defense     800 non-null    int64 
 5   Sp. Atk     800 non-null    int64 
 6   Sp. Def     800 non-null    int64 
 7   Speed       800 non-null    int64 
 8   Generation  800 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 56.4+ KB


Se eliminan columnas "Name", "Legendary" y "#", ya que no serán utilizadas para este ejercicio.

In [ ]:
df.duplicated().sum()


7

In [ ]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Type 1      793 non-null    object
 1   Total       793 non-null    int64 
 2   HP          793 non-null    int64 
 3   Attack      793 non-null    int64 
 4   Defense     793 non-null    int64 
 5   Sp. Atk     793 non-null    int64 
 6   Sp. Def     793 non-null    int64 
 7   Speed       793 non-null    int64 
 8   Generation  793 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 62.0+ KB


In [ ]:
df["Type 1"].value_counts()

Water       111
Normal       98
Grass        70
Bug          68
Psychic      56
Fire         52
Rock         44
Electric     40
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
etiq = LabelEncoder()
pok_label = etiq.fit_transform(df["Type 1"])
pok_m = {index: label for index, label in 
                  enumerate(etiq.classes_)}
pok_m

{0: 'Bug',
 1: 'Dark',
 2: 'Dragon',
 3: 'Electric',
 4: 'Fairy',
 5: 'Fighting',
 6: 'Fire',
 7: 'Flying',
 8: 'Ghost',
 9: 'Grass',
 10: 'Ground',
 11: 'Ice',
 12: 'Normal',
 13: 'Poison',
 14: 'Psychic',
 15: 'Rock',
 16: 'Steel',
 17: 'Water'}

In [ ]:
df["Type 1"] = pok_label
df["Type 1"].value_counts()

17    111
12     98
9      70
0      68
14     56
6      52
15     44
3      40
2      32
10     32
8      32
1      31
13     28
16     27
5      27
11     24
4      17
7       4
Name: Type 1, dtype: int64

Se transforman los datos de la columna "Type 1" a númericos.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 799
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Type 1      793 non-null    int64
 1   Total       793 non-null    int64
 2   HP          793 non-null    int64
 3   Attack      793 non-null    int64
 4   Defense     793 non-null    int64
 5   Sp. Atk     793 non-null    int64
 6   Sp. Def     793 non-null    int64
 7   Speed       793 non-null    int64
 8   Generation  793 non-null    int64
dtypes: int64(9)
memory usage: 62.0 KB


In [ ]:
X = df.drop(columns=["Type 1"])
y = df["Type 1"]

X

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,318,45,49,49,65,65,45,1
1,405,60,62,63,80,80,60,1
2,525,80,82,83,100,100,80,1
3,625,80,100,123,122,120,80,1
4,309,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...,...
795,600,50,100,150,100,150,50,6
796,700,50,160,110,160,110,110,6
797,600,80,110,60,150,130,70,6
798,680,80,160,60,170,130,80,6


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,test_size=0.5)

In [ ]:
print("Muestras de entrenamiento", X_train.shape)
print("Muestras de validacion", X_val.shape)
print("Muestras de test", X_test.shape)

Muestras de entrenamiento (555, 8)
Muestras de validacion (119, 8)
Muestras de test (119, 8)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val =scaler.transform(X_val)

In [ ]:
from sklearn.metrics import recall_score
recall=0
k=1
aux=0
aux1=0
while(recall<=0.2):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  predictions = knn.predict(X_test)
  recall=recall_score(y_test, predictions , average="macro")
  if (recall==aux):
    aux1=aux1+1
  aux=recall
  if (aux1==5):
    k=k+1
    aux1=0
  if (k==len(X_train)):
    break
  print(recall)

0.15107650903498657
0.15107650903498657
0.15107650903498657
0.15107650903498657
0.15107650903498657
0.15107650903498657
0.2094867358708189


In [ ]:
validacion=knn.predict(X_val)

In [ ]:
print("Valores predichos", validacion.tolist())
print("Valores reales", y_val.tolist())
print('Accuracy:',knn.score(X_val, y_val))
print('Recall:',recall_score(validacion, y_val, average="macro"))

Valores predichos [3, 6, 12, 2, 3, 9, 3, 6, 0, 14, 4, 12, 0, 1, 16, 9, 12, 14, 10, 12, 17, 1, 13, 17, 6, 9, 9, 3, 1, 12, 17, 9, 9, 1, 12, 9, 5, 6, 16, 11, 3, 1, 1, 0, 6, 10, 6, 1, 3, 9, 8, 16, 5, 12, 1, 0, 13, 3, 12, 11, 1, 0, 6, 12, 0, 4, 14, 2, 6, 9, 9, 0, 4, 6, 3, 0, 5, 10, 12, 6, 12, 17, 6, 1, 9, 9, 6, 4, 1, 0, 12, 1, 12, 3, 10, 4, 8, 10, 9, 10, 0, 13, 2, 9, 0, 0, 4, 12, 6, 2, 3, 9, 0, 9, 6, 0, 9, 12, 3]
Valores reales [15, 14, 12, 2, 17, 3, 8, 17, 16, 3, 14, 17, 13, 2, 2, 9, 13, 1, 17, 13, 17, 12, 13, 2, 0, 4, 4, 17, 4, 12, 9, 5, 6, 2, 8, 14, 0, 13, 12, 8, 17, 12, 6, 0, 12, 15, 17, 8, 14, 17, 16, 2, 3, 12, 3, 0, 9, 17, 5, 8, 0, 5, 2, 9, 13, 11, 2, 17, 1, 12, 11, 9, 17, 1, 9, 16, 5, 10, 14, 9, 13, 14, 8, 17, 10, 8, 2, 15, 8, 0, 12, 16, 17, 14, 13, 4, 0, 11, 1, 3, 6, 8, 8, 17, 0, 15, 12, 12, 12, 15, 3, 2, 15, 2, 17, 16, 12, 5, 3]
Accuracy: 0.15126050420168066
Recall: 0.13880525472220975


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(classification_report(y_val, validacion, target_names=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17"]))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         5
           3       0.09      0.17      0.12         6
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         0
           6       0.21      0.25      0.23        12
           7       0.00      0.00      0.00         1
           8       0.50      0.25      0.33         4
           9       0.12      0.20      0.15        10
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         0
          12       0.27      0.24      0.25        17
          13       0.00      0.00      0.00         2
          14       0.33      0.07      0.11        15
          15       0.00      0.00      0.00        10
          16       0.00      0.00      0.00         7
          17       0.25    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

* Al no estratificar, se puede haber borrado alguna clase de pokemon.

In [ ]:
pok_m

{0: 'Bug',
 1: 'Dark',
 2: 'Dragon',
 3: 'Electric',
 4: 'Fairy',
 5: 'Fighting',
 6: 'Fire',
 7: 'Flying',
 8: 'Ghost',
 9: 'Grass',
 10: 'Ground',
 11: 'Ice',
 12: 'Normal',
 13: 'Poison',
 14: 'Psychic',
 15: 'Rock',
 16: 'Steel',
 17: 'Water'}

# Ejercicio 3

Generar modelo para predecir "main.disorder" en base a las columnas de examenes.

In [ ]:
filename = "/content/EEG.machinelearing_data_BRMH.csv"
df = pd.read_csv(filename)
df_copy= df
df.head()

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Columns: 1149 entries, no. to COH.F.gamma.r.O1.s.O2
dtypes: float64(1144), int64(1), object(4)
memory usage: 8.3+ MB


In [ ]:
df.isna().sum()

no.                       0
sex                       0
age                       0
eeg.date                  0
education                15
                         ..
COH.F.gamma.p.P4.r.O1     0
COH.F.gamma.p.P4.s.O2     0
COH.F.gamma.q.T6.r.O1     0
COH.F.gamma.q.T6.s.O2     0
COH.F.gamma.r.O1.s.O2     0
Length: 1149, dtype: int64

In [ ]:
df["IQ"].isna().sum()

13

In [ ]:
df["education"].isna().sum()

15

In [ ]:
df = df.drop(columns=["no.", "eeg.date"])
df.head()

,sex,age,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,M,57.0,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,21.518280,26.825048,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,M,37.0,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,11.942516,15.272216,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,M,32.0,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,17.150159,23.608960,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,M,35.0,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,17.364316,13.833701,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,M,36.0,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,21.865556,21.771413,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


In [ ]:
df["IQ"].describe()

count    932.000000
mean     101.580472
std       17.022414
min       49.000000
25%       91.000000
50%      102.000000
75%      114.000000
max      145.000000
Name: IQ, dtype: float64

In [ ]:
df["education"].describe()

count    930.000000
mean      13.438710
std        2.550461
min        0.000000
25%       12.000000
50%       13.000000
75%       16.000000
max       20.000000
Name: education, dtype: float64

In [ ]:
df["IQ"].fillna(101.58, inplace = True) 
df["education"].fillna(13.43, inplace = True) 
df["IQ"].isna().sum()

0

Se rellenan los valores nulos con las medias de ambas columnas.

In [ ]:
df = df.loc[:, "sex":"COH.F.gamma.r.O1.s.O2"].fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Columns: 1147 entries, sex to COH.F.gamma.r.O1.s.O2
dtypes: float64(1144), object(3)
memory usage: 8.3+ MB


Ante la dificultad para encontrar los NaN, se decide rellenar con 0 dichos valores. Se entiende que el examen pudo tener problemas y no se obtuvo un resultado correcto.

In [ ]:
df.duplicated().sum()

0

In [ ]:
df["main.disorder"].value_counts()

Mood disorder                         266
Addictive disorder                    186
Trauma and stress related disorder    128
Schizophrenia                         117
Anxiety disorder                      107
Healthy control                        95
Obsessive compulsive disorder          46
Name: main.disorder, dtype: int64

In [ ]:
df["specific.disorder"].value_counts()

Depressive disorder               199
Schizophrenia                     117
Healthy control                    95
Alcohol use disorder               93
Behavioral addiction disorder      93
Bipolar disorder                   67
Panic disorder                     59
Posttraumatic stress disorder      52
Social anxiety disorder            48
Obsessive compulsitve disorder     46
Acute stress disorder              38
Adjustment disorder                38
Name: specific.disorder, dtype: int64

In [ ]:
df["sex"].value_counts()

M    601
F    344
Name: sex, dtype: int64

In [ ]:
for i in df.index:
  if df["sex"][i]=="M":
    df["sex"][i]=1
  elif df["sex"][i]=="F":
    df["sex"][i]=0


df["sex"].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


1    601
0    344
Name: sex, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
etiq = LabelEncoder()
eeg_label = etiq.fit_transform(df["specific.disorder"])
eeg_m = {index: label for index, label in 
                  enumerate(etiq.classes_)}
eeg_m

{0: 'Acute stress disorder',
 1: 'Adjustment disorder',
 2: 'Alcohol use disorder',
 3: 'Behavioral addiction disorder',
 4: 'Bipolar disorder',
 5: 'Depressive disorder',
 6: 'Healthy control',
 7: 'Obsessive compulsitve disorder',
 8: 'Panic disorder',
 9: 'Posttraumatic stress disorder',
 10: 'Schizophrenia',
 11: 'Social anxiety disorder'}

In [ ]:
df["specific.disorder"].value_counts()

Depressive disorder               199
Schizophrenia                     117
Healthy control                    95
Alcohol use disorder               93
Behavioral addiction disorder      93
Bipolar disorder                   67
Panic disorder                     59
Posttraumatic stress disorder      52
Social anxiety disorder            48
Obsessive compulsitve disorder     46
Acute stress disorder              38
Adjustment disorder                38
Name: specific.disorder, dtype: int64

In [ ]:
df["specific.disorder"] = eeg_label
df["specific.disorder"].value_counts()

5     199
10    117
6      95
2      93
3      93
4      67
8      59
9      52
11     48
7      46
0      38
1      38
Name: specific.disorder, dtype: int64

In [ ]:
X = df.drop(columns=["main.disorder"])
y = df["main.disorder"]

X

,sex,age,education,IQ,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,57.0,13.43,101.58,2,35.998557,21.717375,21.518280,26.825048,26.611516,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,1,37.0,6.00,120.00,2,13.425118,11.002916,11.942516,15.272216,14.151570,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,1,32.0,16.00,113.00,2,29.941780,27.544684,17.150159,23.608960,27.087811,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,1,35.0,18.00,126.00,2,21.496226,21.846832,17.364316,13.833701,14.100954,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,1,36.0,16.00,112.00,2,37.775667,33.607679,21.865556,21.771413,22.854536,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,1,22.0,13.00,116.00,6,41.851823,36.771496,43.671792,36.860889,24.732236,...,82.905657,34.850706,63.970519,63.982003,51.244725,62.203684,62.062237,31.013031,31.183413,98.325230
941,1,26.0,13.00,118.00,6,18.986856,19.401387,27.586436,20.194732,19.407491,...,65.917918,66.700117,44.756285,49.787513,98.905995,54.021304,93.902401,52.740396,92.807331,56.320868
942,1,26.0,16.00,113.00,6,28.781317,32.369230,11.717778,23.134370,26.209302,...,61.040959,27.632209,45.552852,33.638817,46.690983,19.382928,41.050717,7.045821,41.962451,19.092111
943,1,24.0,13.00,107.00,6,19.929100,25.196375,14.445391,16.453456,16.590649,...,99.113664,48.328934,41.248470,28.192238,48.665743,42.007147,28.735945,27.176500,27.529522,20.028446


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,test_size=0.5)

In [ ]:
print("Muestras de entrenamiento", X_train.shape)
print("Muestras de validacion", X_val.shape)
print("Muestras de test", X_test.shape)

Muestras de entrenamiento (661, 1146)
Muestras de validacion (142, 1146)
Muestras de test (142, 1146)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val =scaler.transform(X_val)

In [ ]:
from sklearn.metrics import recall_score
recall=0
k=1
aux=0
aux1=0
while(recall<=0.2):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  predictions = knn.predict(X_test)
  recall=recall_score(y_test, predictions, average="macro")
  if (recall==aux):
    aux1=aux1+1
  aux=recall
  if (aux1==5):
    k=k+1
    aux1=0
  if (k==len(X_train)):
    break
  print(recall)

0.19779650990831113
0.19779650990831113
0.19779650990831113
0.19779650990831113
0.19779650990831113
0.19779650990831113
0.16129251700680272
0.16129251700680272
0.16129251700680272
0.16129251700680272
0.16129251700680272
0.16129251700680272
0.20246968352558414


In [ ]:
validacion=knn.predict(X_val)

In [ ]:
print("Valores predichos", validacion.tolist())
print("Valores reales", y_val.tolist())
print('Accuracy:',knn.score(X_val, y_val))
print('Recall:',recall_score(y_val, validacion, average="macro"))

Valores predichos [1, 0, 0, 5, 5, 0, 0, 3, 6, 0, 3, 3, 3, 0, 1, 6, 3, 0, 3, 3, 0, 5, 0, 2, 1, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 4, 1, 0, 2, 0, 3, 0, 5, 1, 0, 3, 3, 0, 3, 3, 0, 0, 3, 1, 6, 5, 1, 0, 0, 0, 0, 0, 5, 4, 3, 0, 5, 1, 1, 3, 1, 6, 0, 0, 1, 0, 0, 3, 4, 3, 0, 0, 5, 3, 1, 0, 3, 3, 2, 3, 0, 2, 1, 3, 3, 0, 3, 2, 3, 6, 2, 0, 0, 1, 0, 5, 3, 0, 0, 0, 6, 0, 3, 0, 0, 3, 0, 6, 3, 0, 3, 2, 0, 3, 5, 6, 6, 2, 2, 0, 6, 5, 0, 3, 0, 1, 3, 3, 0, 6]
Valores reales [2, 1, 2, 2, 2, 2, 0, 5, 6, 4, 3, 0, 0, 3, 5, 4, 6, 3, 3, 1, 3, 0, 0, 6, 6, 5, 3, 3, 5, 2, 3, 6, 6, 6, 1, 2, 0, 6, 5, 6, 1, 3, 3, 3, 3, 5, 5, 5, 2, 5, 1, 4, 2, 3, 2, 1, 5, 5, 0, 3, 3, 3, 3, 3, 1, 2, 0, 0, 5, 1, 0, 0, 0, 1, 1, 2, 0, 3, 0, 6, 0, 0, 0, 6, 3, 4, 6, 4, 0, 1, 5, 3, 0, 2, 3, 0, 3, 6, 0, 2, 5, 5, 3, 3, 3, 5, 3, 6, 1, 3, 1, 3, 6, 0, 1, 4, 3, 3, 3, 0, 2, 3, 0, 1, 5, 6, 4, 3, 0, 1, 6, 0, 0, 0, 3, 3, 3, 3, 6, 6, 3, 3]
Accuracy: 0.176056338028169
Recall: 0.14323887449716732


In [ ]:
df["main.disorder"].unique()

array(['Addictive disorder', 'Trauma and stress related disorder',
       'Mood disorder', 'Healthy control',
       'Obsessive compulsive disorder', 'Schizophrenia',
       'Anxiety disorder'], dtype=object)

In [ ]:
print(classification_report(y_val, validacion, target_names=df["main.disorder"].unique()))

                                    precision    recall  f1-score   support

                Addictive disorder       0.29      0.62      0.40        29
Trauma and stress related disorder       0.07      0.13      0.09        15
                     Mood disorder       0.22      0.14      0.17        14
                   Healthy control       0.31      0.23      0.26        40
     Obsessive compulsive disorder       0.00      0.00      0.00         5
                     Schizophrenia       0.33      0.05      0.09        19
                  Anxiety disorder       0.44      0.20      0.28        20

                          accuracy                           0.25       142
                         macro avg       0.24      0.20      0.18       142
                      weighted avg       0.28      0.25      0.23       142



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Encontrar 5 examenes más relevantes para predecir "main.disorder".**

In [ ]:
from sklearn.preprocessing import LabelEncoder
etiq = LabelEncoder()
eeg_label = etiq.fit_transform(df["main.disorder"])
eeg_m = {index: label for index, label in 
                  enumerate(etiq.classes_)}
eeg_m

{0: 'Addictive disorder',
 1: 'Anxiety disorder',
 2: 'Healthy control',
 3: 'Mood disorder',
 4: 'Obsessive compulsive disorder',
 5: 'Schizophrenia',
 6: 'Trauma and stress related disorder'}

In [ ]:
df["main.disorder"].value_counts()

Mood disorder                         266
Addictive disorder                    186
Trauma and stress related disorder    128
Schizophrenia                         117
Anxiety disorder                      107
Healthy control                        95
Obsessive compulsive disorder          46
Name: main.disorder, dtype: int64

In [ ]:
df["main.disorder"] = eeg_label
df["main.disorder"].value_counts()

3    266
0    186
6    128
5    117
1    107
2     95
4     46
Name: main.disorder, dtype: int64

In [ ]:
mat_corr = df.corr()

In [ ]:
mat_corr.sort_values(by="main.disorder", ascending=False).head(8)

,age,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
main.disorder,0.161851,-0.005497,-0.170308,1.000000,0.233532,0.084953,0.100162,0.109278,0.082967,0.069904,...,0.012746,-0.019321,-0.016685,0.006407,-0.027909,-0.020032,-0.013575,-0.023797,-0.016191,0.015149
specific.disorder,0.038299,-0.061135,-0.158912,0.233532,1.000000,0.038681,0.048065,0.078722,0.017857,-0.013186,...,-0.011855,0.001102,0.009206,-0.023472,-0.005118,0.001713,-0.036466,0.004997,-0.000871,0.018513
AB.D.beta.g.F8,0.030199,-0.018324,-0.066257,0.174348,0.072593,0.264679,0.282697,0.286928,0.246864,0.235861,...,-0.010070,-0.010506,-0.020808,-0.035726,-0.017536,-0.007777,-0.042262,-0.014463,-0.028952,-0.019748
age,1.000000,-0.160317,0.023315,0.161851,0.038299,0.006552,-0.013299,0.019729,0.007273,-0.003285,...,0.014012,-0.025980,-0.021693,-0.018525,-0.018467,-0.040804,-0.043727,-0.040422,-0.018585,-0.007980
AB.D.beta.f.F4,0.019266,-0.025730,-0.047643,0.160857,0.056473,0.227580,0.245232,0.233016,0.244677,0.246575,...,-0.012178,-0.002250,-0.008509,-0.034494,-0.012736,-0.006919,-0.046586,-0.004856,-0.031031,-0.030924
AB.D.beta.a.FP1,0.008901,-0.040925,-0.055586,0.156035,0.029744,0.306975,0.295474,0.294058,0.264934,0.257395,...,0.003668,-0.002940,-0.002021,-0.012219,-0.013266,0.001892,-0.022743,-0.006951,-0.017301,-0.010976
AB.D.beta.k.C4,0.024016,-0.021488,-0.040008,0.153694,0.028228,0.197724,0.214805,0.207595,0.193198,0.189296,...,-0.005241,-0.000478,0.002055,-0.023448,-0.015015,0.002072,-0.040016,0.003481,-0.034352,-0.017543
AB.D.beta.b.FP2,0.012514,-0.025693,-0.048379,0.148692,0.037885,0.270920,0.339247,0.269941,0.255249,0.253905,...,-0.004052,-0.011793,-0.013271,-0.020064,-0.013742,-0.011792,-0.030729,-0.020195,-0.028396,-0.016856


In [ ]:
df1 = df.loc[:, ["main.disorder", "AB.D.beta.g.F8", "AB.D.beta.f.F4", "AB.D.beta.a.FP1", "AB.D.beta.k.C4", "AB.D.beta.b.FP2"]]
df1.head()

,main.disorder,AB.D.beta.g.F8,AB.D.beta.f.F4,AB.D.beta.a.FP1,AB.D.beta.k.C4,AB.D.beta.b.FP2
0,0,8.126729,12.645652,11.202471,10.551489,11.111987
1,0,14.097111,26.931780,30.423142,25.091939,23.859927
2,0,6.746350,6.397807,13.799929,10.571053,10.315396
3,0,4.940311,6.528162,6.429691,5.223149,6.212037
4,0,10.554364,14.348928,10.147936,17.314899,11.017907


In [ ]:
X = df1.drop(columns=["main.disorder"])
y = df1["main.disorder"]

X

,AB.D.beta.g.F8,AB.D.beta.f.F4,AB.D.beta.a.FP1,AB.D.beta.k.C4,AB.D.beta.b.FP2
0,8.126729,12.645652,11.202471,10.551489,11.111987
1,14.097111,26.931780,30.423142,25.091939,23.859927
2,6.746350,6.397807,13.799929,10.571053,10.315396
3,4.940311,6.528162,6.429691,5.223149,6.212037
4,10.554364,14.348928,10.147936,17.314899,11.017907
...,...,...,...,...,...
940,7.630762,7.630037,9.969296,6.830363,5.815453
941,7.990898,8.878700,8.358988,9.045869,8.504071
942,8.191237,21.678269,16.310102,9.026125,16.320458
943,11.122609,11.935914,11.177623,11.413014,11.777143


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.3, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val,test_size=0.5, stratify=y_val)

In [ ]:
print("Muestras de entrenamiento", X_train.shape)
print("Muestras de validacion", X_val.shape)
print("Muestras de test", X_test.shape)

Muestras de entrenamiento (661, 5)
Muestras de validacion (142, 5)
Muestras de test (142, 5)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val =scaler.transform(X_val)

In [ ]:
from sklearn.metrics import recall_score
recall=0
k=1
aux=0
aux1=0
while(recall<=0.15):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  predictions = knn.predict(X_test)
  recall=recall_score(y_test, predictions, average="macro")
  if (recall==aux):
    aux1=aux1+1
  aux=recall
  if (aux1==5):
    k=k+1
    aux1=0
  if (k==len(X_train)):
    break
  print(recall)

0.13507727652464493
0.13507727652464493
0.13507727652464493
0.13507727652464493
0.13507727652464493
0.13507727652464493
0.12290995345506624
0.12290995345506624
0.12290995345506624
0.12290995345506624
0.12290995345506624
0.12290995345506624
0.13237707363647214
0.13237707363647214
0.13237707363647214
0.13237707363647214
0.13237707363647214
0.13237707363647214
0.15839748179973742


In [ ]:
validacion=knn.predict(X_val)

In [ ]:
print("Valores predichos", validacion.tolist())
print("Valores reales", y_val.tolist())
print('Accuracy:',knn.score(X_val, y_val))
print('Recall:',recall_score(y_val, validacion, average="macro"))

Valores predichos [1, 0, 0, 5, 5, 0, 0, 3, 6, 0, 3, 3, 3, 0, 1, 6, 3, 0, 3, 3, 0, 5, 0, 2, 1, 0, 3, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 4, 1, 0, 2, 0, 3, 0, 5, 1, 0, 3, 3, 0, 3, 3, 0, 0, 3, 1, 6, 5, 1, 0, 0, 0, 0, 0, 5, 4, 3, 0, 5, 1, 1, 3, 1, 6, 0, 0, 1, 0, 0, 3, 4, 3, 0, 0, 5, 3, 1, 0, 3, 3, 2, 3, 0, 2, 1, 3, 3, 0, 3, 2, 3, 6, 2, 0, 0, 1, 0, 5, 3, 0, 0, 0, 6, 0, 3, 0, 0, 3, 0, 6, 3, 0, 3, 2, 0, 3, 5, 6, 6, 2, 2, 0, 6, 5, 0, 3, 0, 1, 3, 3, 0, 6]
Valores reales [2, 1, 2, 2, 2, 2, 0, 5, 6, 4, 3, 0, 0, 3, 5, 4, 6, 3, 3, 1, 3, 0, 0, 6, 6, 5, 3, 3, 5, 2, 3, 6, 6, 6, 1, 2, 0, 6, 5, 6, 1, 3, 3, 3, 3, 5, 5, 5, 2, 5, 1, 4, 2, 3, 2, 1, 5, 5, 0, 3, 3, 3, 3, 3, 1, 2, 0, 0, 5, 1, 0, 0, 0, 1, 1, 2, 0, 3, 0, 6, 0, 0, 0, 6, 3, 4, 6, 4, 0, 1, 5, 3, 0, 2, 3, 0, 3, 6, 0, 2, 5, 5, 3, 3, 3, 5, 3, 6, 1, 3, 1, 3, 6, 0, 1, 4, 3, 3, 3, 0, 2, 3, 0, 1, 5, 6, 4, 3, 0, 1, 6, 0, 0, 0, 3, 3, 3, 3, 6, 6, 3, 3]
Accuracy: 0.176056338028169
Recall: 0.14323887449716732


In [ ]:
print(classification_report(y_val, validacion, target_names=["0" ,"1", "2", "3", "4", "5", "6"]))

              precision    recall  f1-score   support

           0       0.16      0.32      0.22        28
           1       0.12      0.12      0.12        16
           2       0.22      0.13      0.17        15
           3       0.22      0.20      0.21        40
           4       0.00      0.00      0.00         7
           5       0.18      0.12      0.14        17
           6       0.18      0.11      0.13        19

    accuracy                           0.18       142
   macro avg       0.16      0.14      0.14       142
weighted avg       0.18      0.18      0.17       142

